In [1]:
import time

import torch
from torch import Tensor
torch.manual_seed(0)
from torch import nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()

def closure():
        optimizer.zero_grad()
        y_hat = model.forward(X_train)
        loss = criterion(y_hat, y_train)
        loss.backward()
        return loss
    
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

import numpy as np
from sklearn.model_selection import GridSearchCV

from copy import deepcopy

data = pd.read_csv('Proj1ReorderedRound.csv')

import warnings
warnings.filterwarnings('ignore')

In [2]:
X = data.drop('Condition', axis=1).values
y = data['Condition'].values

y = LE.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train = stdscaler.fit_transform(X_train)
X_test = stdscaler.transform(X_test)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [3]:
class ANN(nn.Module):
    def __init__(self,af, array):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features = 12, out_features = array[0])
        self.fc2 = nn.Linear(in_features = array[0], out_features = array[1])
        self.fc3 = nn.Linear(in_features = array[1], out_features = array[2])
        self.output = nn.Linear(in_features=array[2], out_features=3)
        
        self.AF = af
 
    def forward(self, x):
            
        if (af == 'sigmoid'):
            x = F.sigmoid(self.fc1(x))
            x = F.sigmoid(self.fc2(x))
            x = F.sigmoid(self.fc3(x))
            x = self.output(x)
            
        if (af == 'tanh'):
            x = F.tanh(self.fc1(x))
            x = F.tanh(self.fc2(x))
            x = F.tanh(self.fc3(x))
            x = self.output(x)
        
        if (af == 'relu'):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = self.output(x)
            
        if (af == 'leaky_relu'):
            x = F.leaky_relu(self.fc1(x))
            x = F.leaky_relu(self.fc2(x))
            x = F.leaky_relu(self.fc3(x))
            x = self.output(x)
            
        if (af == 'elu'):
            x = F.elu(self.fc1(x))
            x = F.elu(self.fc2(x))
            x = F.elu(self.fc3(x))
            x = self.output(x)
            
        if (af == 'softmax'):
            x = F.softmax(self.fc1(x))
            x = F.softmax(self.fc2(x))
            x = F.softmax(self.fc3(x))
            x = self.output(x)
            
        if (af == 'hardswish'):
            x = F.hardswish(self.fc1(x))
            x = F.hardswish(self.fc2(x))
            x = F.hardswish(self.fc3(x))
            x = self.output(x)
            
        if (af == 'gelu'):
            x = F.gelu(self.fc1(x))
            x = F.gelu(self.fc2(x))
            x = F.gelu(self.fc3(x))
            x = self.output(x)
            
        if (af == 'selu'):
            x = F.selu(self.fc1(x))
            x = F.selu(self.fc2(x))
            x = F.selu(self.fc3(x))
            x = self.output(x)
        
        return x
    
alpha = 0.1
criterion = nn.CrossEntropyLoss()

In [7]:
initiator = 0

two_thirds = [12,8,5,3,2]

ones = []
threes = []
halves = []
fulls = []
desc = []
asc = []

config_list = []


for i in range(3):
    ones.append(1)
    threes.append(3)
    halves.append(6)
    fulls.append(12)
    desc.append(two_thirds[i])
    asc.append(two_thirds[:3][-1-i])

configDF = pd.DataFrame({'ones':ones,'threes':threes,'halves':halves,'fulls':fulls,'desc':desc,'asc':asc})
print('# of hidden layers : ', 3)
configDF

# of hidden layers :  3


,ones,threes,halves,fulls,desc,asc
0,1,3,6,12,12,5
1,1,3,6,12,8,8
2,1,3,6,12,5,12


In [5]:
af_list = ['sigmoid','tanh','relu','leaky_relu','elu','softmax','hardswish','gelu','selu']
af = af_list[8]   # i = 0 to 8

In [6]:
time_list = []

epochs = 100
print('Activation function : ', af)
print('Epochs : ', epochs)

LossDF = pd.DataFrame(columns = ['config','i','j','nodes','initial','midway','final'])
index = 0

if (initiator == 0):    #checks if this cell has been run before...
    FML = []   # Full Model List : stores initial model weights and bias for future reference
      
for config in configDF.columns:
    Start = time.time()     #start time
    
    to_add = [config]
    
    Arr = configDF[config]
    to_add.append(Arr[0])
    to_add.append(Arr[1])
    to_add.append(Arr.sum())
    
    if (initiator == 0):        # if this cell wasnt run before, store the starting weights and bias for future ref
        model = ANN(af,Arr)
        dummy = deepcopy(model)
        FML.append(dummy)
        
    elif (initiator == 1):      # otherwise, use the same starting params...
        model = deepcopy(FML[index])
        setattr(model, 'array', Arr)    #... while overwriting the hidden layer configuration
        
    optimizer = torch.optim.SGD(model.parameters(), lr=alpha) # SGD
    
    for e in range(epochs):
        y_hat = model.forward(X_train)
        loss = criterion(y_hat, y_train)
                
        if e == 0:
            to_add.append(loss.detach().numpy())
                    
        if e == 49:
            to_add.append(loss.detach().numpy())
 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
    to_add.append(loss.detach().numpy())
            
    LossDF.loc[index] = to_add
    index += 1
    
    
    End = time.time()
    dura = End - Start
    time_list.append(dura)
    
    print('\nComputation for',config,'complete')
    print('Time taken : ', np.round(dura,3),'s')

initiator = 1

print('\nActivation function : ', af)

print('\nTotal time taken :',np.round(np.sum(time_list),3),'s')

print('average loss = ', np.round(LossDF.mean()[-1],5))
print('average loss (without ones)= ', np.round(LossDF.drop(0).mean()[-1],8))

LossDF

Activation function :  selu
Epochs :  100

Computation for ones complete
Time taken :  0.054 s

Computation for threes complete
Time taken :  0.057 s

Computation for halves complete
Time taken :  0.06 s

Computation for fulls complete
Time taken :  0.071 s

Computation for desc complete
Time taken :  0.064 s

Computation for asc complete
Time taken :  0.063 s

Activation function :  selu

Total time taken : 0.368 s
average loss =  0.15648
average loss (without ones)=  0.10294577


,config,i,j,nodes,initial,midway,final
0,ones,1,1,3,1.2169077,0.68365836,0.42413065
1,threes,3,3,9,1.2150943,0.41333014,0.15520404
2,halves,6,6,18,1.0735034,0.31963876,0.10060681
3,fulls,12,12,36,1.0895646,0.19618917,0.056515258
4,desc,12,8,25,1.2006211,0.3361293,0.112017505
5,asc,5,8,25,1.1018398,0.23035742,0.09038521
